In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import cv2
from utils import DeepSortMock, drawDetection, random_bbox
import pickle

In [3]:
class Identity():
    def __init__(self, ID, memory_length = 45): # Frames):
        self.ID = ID
        self.age = 0
        self.name = str(ID) 
        self.memory_length = memory_length
        self.__reference_images = []
    def push_image(self,frame):
        self.__reference_images.append(frame)
        self.__reference_images = self.__reference_images[-self.memory_length:]
    def get_images(self):
        return self.__reference_images
    def get_latest_image(self):
        return self.__reference_images[-1]

In [5]:
data_files = ["grandma_A"]
feeds = [cv2.VideoCapture(f"raw_data/videos/{file}.mp4") for file in data_files]
detectors = [DeepSortMock(f"raw_data/tracks/{file}.pb") for file in data_files]
# seperators = [SVC(kernel="poly", probability = True) for _ in data_files]

feed_identities = [dict() for _ in feeds] # TODO: Store in a queue instead

# DEBUG 
output_frames = []
clean_detections = []

frames_left = True
frame_num = 0 
try:
    while frames_left:
     
        clean_detections.append([])
        frame_num += 1
        output_frames = [ ]
        for i, (feed, detector, identities) in enumerate(zip(feeds, detectors, feed_identities)):
            updated_ids = set()
            frames_left, frame = feed.read()
 
            if not frames_left: break

            people = detector.update(frame)
            
            # Grab Positive Person images to train seperators  
            for person in people:
                p1, p2, cat, ID = person
                updated_ids.add(ID)  
           
                if ID in [6, 9]:
                    ID = 0
                if ID in [5, 7, 10, 11]:
                    ID = 1

                clean_detections[-1].append((p1, p2, cat, ID))
                frame = drawDetection(frame, (p1, p2, cat, ID), info = frame_num)
        
            output_frames.append(frame) 
            
        ##################################################
        try:
            cv2.imshow("Joined Product",
                np.concatenate(tuple(output_frame for output_frame in output_frames), axis = 1))
            cv2.waitKey(1)
        except :
            pass
        cv2.waitKey(1)
                
finally:
    [cap.release() for cap in feeds]
    out = open(b"raw_data/tracks/grandma_A.cleaned.pbs", "wb")
    pickle.dump(clean_detections, out)
    cv2.destroyAllWindows()